In [27]:
import sqlite3
import pandas as pd

import conf


connection = sqlite3.connect(conf.DB_NAME)
cursor = connection.cursor()

In [28]:
df = pd.read_sql(sql="SELECT * FROM analytics", con=connection)

In [29]:
df.head(3)

,user,purchase_number,total_sales,currency,favorite_segment,M0_cohort,M1_retention,lifespan_days
0,65326324392533,1,250.0,eur,meal,2023-05,0,0
1,168982159910603,1,650.0,eur,snack,2023-08,0,0
2,219034330643057,3,2020.0,eur,meal,2022-11,1,359


In [74]:
# Calculate purchase frequency (weekly)
pd.set_option('use_inf_as_na', True)
df['weekly_freq'] = (df.purchase_number / (df.lifespan_days // 7)).fillna(0)
df.weekly_freq.quantile([0.25,0.5,0.56,0.57,0.75,0.93,0.98])

/var/folders/d7/jvw6zrm531q4bhp6_kft_0740000gn/T/ipykernel_29645/3009279054.py:2: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.set_option('use_inf_as_na', True)


0.25    0.000000
0.50    0.000000
0.56    0.000000
0.57    0.044444
0.75    0.150000
0.93    0.480000
0.98    1.000000
Name: weekly_freq, dtype: float64

In [75]:
# Some frequency statistics
# 1. Only 2% of users are making 1 and more orders per week
# 2. Around 7% of users are making 1 order every two weeks
# 3. Around 56% of users made only 1 order at all

In [76]:
# Average order value analysis
df['avg_order'] = df.total_sales / df.purchase_number
df.avg_order.quantile([0.25,0.5,0.75])

0.25    400.000000
0.50    590.000000
0.75    807.223214
Name: avg_order, dtype: float64

In [80]:
# Some order statistics
# 1. Expected order value in "average" can be considered as 590 euro
# 2. Real average is much more (838 euro) because of outliers, but it's less informative when it comes to make up some expectations
# 3. Only 25% of users have their average order amount more than 800 euro

df.avg_order.mean()

838.8143674742032

In [89]:
# Users' lifespan analysis
df.lifespan_days.quantile([0.25,0.5,0.55,0.6,0.775, 0.9])

0.250      0.0
0.500      0.0
0.550      1.0
0.600     24.0
0.775    159.0
0.900    282.0
Name: lifespan_days, dtype: float64

In [90]:
# Some lifespan insights
# 1. Around 55% of users made only 1 order, their lifespain is 0 (as it shown before)
# 2. From those who made more than 1 order the median lifespan (0.775 quantile) is around 159 days
# 3. 10% of users made some orders within a period of more than 280 days